The linear MPC problem can be formulated as follows:
$$
\begin{aligned}
    \min_{\bar{x}, \bar{u}} \quad & \sum_{k=0}^{N-1} \frac{1}{2} x_k^\top Q x_k + \frac{1}{2} u_k^\top R u_k + \frac{1}{2} u_N^\top P u_N \\
    \text{s.t.} \quad & x_{k+1} = A x_k + B u_k, \;  k = 0, \dots, N-1, \\
    & \underline{c} \leq D x_k \leq  \bar{c}, \quad\quad\quad k = 1, \dots, N, \\
    & u_{\text{lb}} \leq u \leq u_{\text{ub}}, \quad\quad\;\; k = 0, \dots, N-1.
\end{aligned}
$$
Here: 
- $Q$, $R$, $P$: cost matrices. 
- $\bar{x} := (x_0, \dots, x_{N})$: state sequence. 
- $\bar{u} := (u_0, \dots, u_{N-1})$: control sequence. 
- $N$: prediction horizon. 
- $u_{\text{lb}}$ and $\underline{c}$: lower bounds.
- $u_{\text{ub}}$ and $\bar{c}$: upper bounds.

## Condensed Formulation

We write the predicted states in matrix form:

$$
\begin{bmatrix}
x_0 \\ x_1 \\ \vdots \\ x_N
\end{bmatrix} = T x_0 + S 
\begin{bmatrix}
u_0 \\ u_1 \\ \vdots \\ u_{N-1}
\end{bmatrix},
$$
where
$$
T := \begin{bmatrix}
I \\ A \\ A^2 \\ \vdots \\ A^N \\
\end{bmatrix}, \quad
S := \begin{bmatrix}
0 & 0 & 0 & \cdots & 0 \\
B & 0 & 0 & \cdots & 0 \\
AB & B & 0 & \cdots & 0 \\
\vdots & \vdots & \vdots & \ddots & 0 \\
A^{N-1}B & A^{N-2}B & A^{N-3}B & \cdots & B
\end{bmatrix}.
$$

We define $\tilde{T}$ and $\tilde{S}$ as:
$$
\begin{bmatrix}
x_0 \\ x_1 \\ \vdots \\ x_{N-1}
\end{bmatrix} = \tilde{T} x_0 + \tilde{S} 
\begin{bmatrix}
u_0 \\ u_1 \\ \vdots \\ u_{N-1}
\end{bmatrix}.
$$

We want to rewrite constraints in matrix form: 
$\blue{G \bar{u} \leq g}$. Let us derive it step by step.

$$
\underbrace
{\begin{cases}
    D \underbrace{(A x_k + B u_k)}_{x_{k+1}} \leq \bar{c}, &k = 0, \dots, N-1, \\
   -D (A x_k + B u_k) \leq -\underline{c}, &k = 0, \dots, N-1, \\
    I u_k \leq u_{\text{ub}}, &k = 0, \dots, N-1, \\
   -I u_k \leq -u_{\text{lb}}, &k = 0, \dots, N-1.
\end{cases}}_{} \\ 
\tilde{D} x_k +  \tilde{E} u_k \leq \tilde{b},\; k = 0, \dots, N-1
$$

$\tilde{D}, \tilde{E}$ and $\tilde{c}$ are derived as:
$$
\tilde{D} = \begin{bmatrix}
    D A \\
    -D A \\
    0 \\
    0
\end{bmatrix},\;
\tilde{E} = \begin{bmatrix}
    D B \\
    -D B \\
    I \\
    -I 
\end{bmatrix},\;
\tilde{b} = \begin{bmatrix}
    \bar{c} \\
    -\underline{c}\\
    u_{\text{ub}} \\
    -u_{\text{lb}}
\end{bmatrix}. \;
$$

$$
\begin{aligned}
    \tilde{D} x_0 +  \tilde{E} u_0 &\leq \tilde{b} \\
    \tilde{D} x_1 +  \tilde{E} u_1 &\leq \tilde{b} \\ 
    &\vdots \\
    \tilde{D} x_{N-1} +  \tilde{E} u_{N-1} &\leq \tilde{b}
\end{aligned} \quad \implies \quad
\bar{D} \begin{bmatrix}
    x_0 \\ x_1 \\ \vdots \\ x_{N-1} 
\end{bmatrix} + \bar{E}
\begin{bmatrix}
    u_0 \\ u_1 \\ \vdots \\ u_{N-1} 
\end{bmatrix} \leq \bar{b}
$$


$$
\begin{aligned}
    \bar{D} \left( \tilde{T}x_0 + \tilde{S} \bar{u} \right) + \bar{E} \bar{u} \leq \bar{b} \\
    \underbrace{\left( \bar{D}\tilde{S} + \bar{E} \right)}_{G} \bar{u} \leq 
    \underbrace{\bar{b} - \bar{D} \tilde{T} x_0}_{g}
\end{aligned}
$$

In [ ]:
def gen_prediction_matrices(Ad, Bd, N):
    dim_x = Ad.shape[0]
    dim_u = Bd.shape[1]
    
    T = np.zeros(((dim_x * (N + 1), dim_x)))
    S = np.zeros(((dim_x * (N + 1), dim_u * N)))
    
    # Condensing
    power_matricies = []    # power_matricies = [I, A, A^2, ..., A^N]
    power_matricies.append(np.eye(dim_x))
    for k in range(N):
        power_matricies.append(power_matricies[k] @ Ad)
    
    for k in range(N + 1):
        T[k * dim_x: (k + 1) * dim_x, :] = power_matricies[k]
        for j in range(N):
            if k > j:
                S[k * dim_x:(k + 1) * dim_x, j * dim_u:(j + 1) * dim_u] = power_matricies[k - j - 1] @ Bd
                
    return T, S

In [ ]:
def gen_cost_matrices(Q, R, P, T, S, x0, N):
    dim_x = Q.shape[0]
    
    Q_bar = np.zeros(((dim_x * (N + 1), dim_x * (N + 1))))
    Q_bar[-dim_x:, -dim_x:] = P
    Q_bar[:dim_x * N, :dim_x * N] = np.kron(np.eye(N), Q) 
    R_bar = np.kron(np.eye(N), R)
    
    H = S.T @ Q_bar @ S + R_bar
    h = S.T @ Q_bar @ T @ x0
    
    H = 0.5 * (H + H.T) # Ensure symmetry!
    
    return H, h

In [ ]:
def gen_constraint_matrices(A, B, T, S,
                            D, c_lb, c_ub, 
                            u_lb, u_ub, 
                            N):

    return G, g

In [14]:
import numpy as np

# nu = 2
# N = 3
# u_lb = np.array([-1, -2])
# u_ub = np.array([1, 2])

nu = 1
N = 3
u_lb = np.array([-1])
u_ub = np.array([2])

Iu = np.eye(nu * N)
Gu = np.vstack([Iu, -Iu])
gu = np.hstack([
    np.kron(np.ones(N), u_ub),
   -np.kron(np.ones(N), u_lb)
])

print(Gu)
print(gu)


(3,)
[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [-1. -0. -0.]
 [-0. -1. -0.]
 [-0. -0. -1.]]
[2. 2. 2. 1. 1. 1.]
